#  TRAFFIC SIGN DETECTION USING CONVOLUTIONAL NEURAL NETWORK

- Developed a Deep Convolutional Neural Network Model that classifies the images of traffic signs.
- Trained the model using the German Traffic Sign Dataset.
- Implemented the Convolutional Neural Network Model based on **LeNet Architecture**.
- The model will classify the images of traffic signs into 43 different classes.
- Classes are as listed below:

    - 0 - Speed limit (20km/h)
    - 1 - Speed limit (30km/h)
    - 2 - Speed limit (50km/h)
    - 3 - Speed limit (60km/h)
    - 4 - Speed limit (70km/h)
    - 5 - Speed limit (80km/h)
    - 6 - End of speed limit (80km/h)
    - 7 - Speed limit (100km/h)
    - 8 - Speed limit (120km/h)
    - 9 - No passing
    - 10 - No passing for vehicles over 3.5 metric tons
    - 11 - Right-of-way at the next intersection
    - 12 - Priority road
    - 13 - Yield
    - 14 - Stop
    - 15 - No vehicles
    - 16 - Vehicles over 3.5 metric tons prohibited
    - 17 - No entry
    - 18 - General caution
    - 19 - Dangerous curve to the left
    - 20 - Dangerous curve to the right
    - 21 - Double curve
    - 22 - Bumpy road
    - 23 - Slippery road
    - 24 - Road narrows on the right
    - 25 - Road work
    - 26 - Traffic signals
    - 27 - Pedestrians
    - 28 - Children crossing
    - 29 - Bicycles crossing
    - 30 - Beware of ice/snow
    - 31 - Wild animals crossing
    - 32 - End of all speed and passing limits
    - 33 - Turn right ahead
    - 34 - Turn left ahead
    - 35 - Ahead only
    - 36 - Go straight or right
    - 37 - Go straight or left
    - 38 - Keep right
    - 39 - Keep left
    - 40 - Roundabout mandatory
    - 41 - End of no passing
    - 42 - End of no passing by vehicles over 3.5 metric tons

# STEP 1: IMPORTING LIBRARIES AND DATASET

# (i) Importing Python Libraries  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import random
import csv

# (ii) Importing Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Dataset is mainly divided into three categories -> Training, Validation and Testing set

with open("/content/drive/MyDrive/ColabNotebooks/proj/traffic-signs-data/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("/content/drive/MyDrive/ColabNotebooks/proj/traffic-signs-data/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("/content/drive/MyDrive/ColabNotebooks/proj/traffic-signs-data/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/proj/traffic-signs-data/train.p'

In [ ]:
# Mapping ClassID to Traffic sign names

signs = []
with open('/content/drive/MyDrive/ColabNotebooks/proj/signnames.csv', 'r') as csvfile:
    signnames = csv.reader(csvfile, delimiter=',')
    next(signnames,None)
    for row in signnames:
        signs.append(row[1])
    csvfile.close()

In [ ]:
#Dividing our datasets into features and labels

X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# Number of training examples
n_train = X_train.shape[0]

# Number of testing examples
n_test = X_test.shape[0]

# Number of validation examples.
n_validation = X_validation.shape[0]

# Shape of a traffic sign image
image_shape = X_train[0].shape

# Number of unique classes/labels in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training images: ", n_train)
print("Number of testing images: ", n_test)
print("Number of validation images: ", n_validation)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

# STEP 2: IMAGE EXPLORATION

- Sample Images from the dataset

In [ ]:
i = 21 #Random number
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 34
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 91
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 130
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 136
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 165
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 154
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
i = 172
plt.imshow(X_test[i])
signs[ y_test[i]]

In [ ]:
#This function plots a histogram of the input data.

def histogram_plot(dataset, label):

    hist, bins = np.histogram(dataset, bins=n_classes)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    plt.xlabel(label)
    plt.ylabel("Image count")
    plt.show()

In [ ]:
# Plotting histograms of the count of each sign
histogram_plot(y_train, "Training images")
histogram_plot(y_test, "Testing images")
histogram_plot(y_validation, "Validation images")

# STEP 3: DATA PREPROCESSING

# (i) Shuffling the data

- In general, we shuffle the training data to increase randomness and variety in training dataset, so that after training, the model is more stable. We don't want our model to learn any kind of specific order/pattern of images.

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

# (ii) Grayscaling

- Converting the coloured images to grayscaled images. Here, the average pixel values of three channels (RGB) is taken and a single channel is created. The dimensions of image (i.e. 32 x 32) is kept same.

In [ ]:
X_train_gray = np.sum(X_train/3, axis=3, keepdims=True)
X_test_gray  = np.sum(X_test/3, axis=3, keepdims=True)
X_validation_gray  = np.sum(X_validation/3, axis=3, keepdims=True)

# (iii) Normalization

- Normalization of image pixel values is done. This process makes the computation faster and the neural network learns more efficiently.

In [ ]:
X_train_gray_norm = (X_train_gray - 128)/128
X_test_gray_norm = (X_test_gray - 128)/128
X_validation_gray_norm = (X_validation_gray - 128)/128

In [ ]:
X_train_gray.shape

In [ ]:
# Visualising an image before and after the data preprocessing

i = 45  #Random number
plt.imshow(X_train[i])
print('Original Image')
signs[ y_train[i]]

In [ ]:
plt.imshow(X_train_gray_norm[i].squeeze(), cmap='gray')
print('Grayscaled Normalized Image')
signs[ y_train[i]]

# STEP 4: MODEL TRAINING

The LeNet Architecture Model consists of the following layers:

- STEP 1: THE FIRST CONVOLUTIONAL LAYER #1
    - Input = 32x32x1
    - Output = 28x28x6
    - Used a 5x5 Filter with output depth of 6
    - Stride is the amount by which the filter/kernel is shifted when the filter is passed over the image
    - Output = (Input-Filter+1)/Stride => (32-5+1)/1 = 28
    - Apply a RELU (Rectified Linear Unit) Activation function to the output
    - Max Pooling for input, Input = 28x28x6 and Output = 14x14x6


- STEP 2: THE SECOND CONVOLUTIONAL LAYER #2
    - Input = 14x14x6
    - Output = 10x10x16
    - Used a 5x5 Filter with output depth of 16
    - Layer 2: Convolutional layer with Output = 10x10x16
    - Output = (Input-filter+1)/strides => 14-5+1/1 = 10
    - Apply a RELU Activation function to the output
    - Max Pooling with Input = 10x10x16 and Output = 5x5x16
    

- STEP 3: FLATTENING THE NETWORK
    - Flatten the network with Input = 5x5x16 and Output = 400
    

- STEP 4: FULLY CONNECTED LAYER 1
    - Layer 3: Fully Connected layer with Input = 400 and Output = 120
    - Apply a RELU Activation function to the output
    

- STEP 5: FULLY CONNECTED LAYER 2
    - Layer 4: Fully Connected Layer with Input = 120 and Output = 84
    - Apply a RELU Activation function to the output
    

- STEP 6: FULLY CONNECTED LAYER 3
    - Layer 5: Fully Connected layer with Input = 84 and Output = 43
    

In [ ]:
# Importing the required Python libraries for building Convolutional Neural Network

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

In [ ]:
image_shape = X_train_gray[i].shape

In [ ]:
# Building the LeNet Architecture CNN model in a sequential manner/fashion.
cnn_model = Sequential()

# Convolution Layer 1
cnn_model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(32,32,1)))

# Pooling/Subsampling Layer 1
cnn_model.add(MaxPooling2D())

# Dropout(Regularization Technique)
cnn_model.add(Dropout(0.4))



# Convolution Layer 2
cnn_model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Pooling/Subsampling Layer 2
cnn_model.add(MaxPooling2D())

# Dropout(Regularization Technique)
cnn_model.add(Dropout(0.4))



# Flattening the network
cnn_model.add(Flatten())

# Hidden Layer 1 (Fully connected layer)
cnn_model.add(Dense(units=120, activation='relu'))

# Hidden Layer 2 (Fully connected layer)
cnn_model.add(Dense(units=84, activation='relu'))

# Output Layer with 43 Classes
cnn_model.add(Dense(units=43, activation = 'softmax'))

In [ ]:
# Compiling the model and training the model

cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])
# Changed 'lr' to 'learning_rate' within the Adam optimizer constructor.

In [ ]:
# Fitting the training dataset in the CNN model

history = cnn_model.fit(X_train_gray_norm,
                        y_train,
                        batch_size=500,
                        epochs=50, # Changed 'nb_epoch' to 'epochs'
                        verbose=1,
                        validation_data = (X_validation_gray_norm,y_validation))

# STEP 5: MODEL EVALUATION

In [ ]:
#Testing/Evaluating the Model with testing dataset which contains over 12000 images

score = cnn_model.evaluate(X_test_gray_norm, y_test)
print('Test Accuracy : {:.2f}'.format(score[1] * 100))

In [ ]:
#Checking the keys in history
history.history.keys()

In [ ]:
#Plotting Training and Validation Accuracy vs Epoch number

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
#Plotting Training and Validation Loss vs Epoch number

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np

# Getting the predictions for the test data
predicted_classes = np.argmax(cnn_model.predict(X_test_gray_norm), axis=-1)
# Use predict() and argmax() instead of predict_classes()

# Actual classes of the test data
y_true_classes = y_test

In [ ]:
# Plotting the confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true_classes, predicted_classes)
plt.figure(figsize = (25,25))
sns.heatmap(cm, annot=True)

Observation :
We observe some clusters in the confusion matrix above. It turns out that the various speed limits are sometimes misclassified among themselves. Similarly, traffic signs with traingular shape are misclassified among themselves. We can further improve on the model using hierarchical CNNs to first identify broader groups (like speed signs) and then have CNNs to classify finer features (such as the actual speed limit).

In [ ]:
#Visualing the results of CNN model by comparing the Predicted class and Actual class of first 25 images from test dataset

L = 5
W = 5
fig, axes = plt.subplots(L, W, figsize = (20,20))
axes = axes.ravel()

for i in np.arange(0, L * W):
    axes[i].imshow(X_test[i])
    axes[i].set_title("Predicted Class ={}\n True Class={} ".format(predicted_classes[i], y_true_classes[i] ))
    axes[i].axis('off')

plt.subplots_adjust(wspace=1)